In [1]:
from llava.model.builder import load_pretrained_model
from PIL import Image

/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


[2025-02-18 15:33:42,178] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/r11kaijun/anaconda3/envs/llava-med/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/r11kaijun/anaconda3/envs/llava-med/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
import json
import os

In [3]:
from llava.constants import DEFAULT_IMAGE_TOKEN, IMAGE_TOKEN_INDEX, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.mm_utils import (
    KeywordsStoppingCriteria,
    # get_model_name_from_path,
    process_images,
    tokenizer_image_token,
)
from llava.conversation import SeparatorStyle, conv_templates
import torch

In [4]:
def load_model(model_path, model_base, model_name):
    tokenizer, model, image_processor, context_len = load_pretrained_model(
        model_path=model_path,
        model_base=model_base,
        model_name=model_name,
        device='cuda',
    )
    vision_tower = model.get_vision_tower()
    if not vision_tower.is_loaded:
        vision_tower.load_model()
    vision_tower.to(device="cuda", dtype=torch.float16)
    model.model.mm_projector.to(device="cuda", dtype=torch.float16)
    model.to(device="cuda", dtype=torch.float16)
    image_processor = vision_tower.image_processor
    if hasattr(model.config, "max_sequence_length"):
        context_len = model.config.max_sequence_length
    else:
        context_len = 2048

    return tokenizer, model, image_processor, context_len

In [10]:
def load_base_model():
    return load_model(
        model_path="microsoft/llava-med-v1.5-mistral-7b",
        model_base="",
        model_name="microsoft/llava-med-v1.5-mistral-7b",
    )

In [6]:
CONV_MODE = "mistral_instruct"
def create_prompt(prompt: str):
    conv = conv_templates[CONV_MODE].copy()
    roles = conv.roles
    # prompt = DEFAULT_IMAGE_TOKEN + "\n" + prompt
    conv.append_message(roles[0], prompt)
    conv.append_message(roles[1], None)
    return conv.get_prompt(), conv

In [7]:
def get_prediction(model, tokenizer, image_processor, image_url: str, question: str):
    # print("image_url:", image_url)
    image = Image.open(image_url)
    image_tensor = process_images([image], image_processor, model.config)[0]
    # print("image_tensor:", image_tensor)

    question = question.replace(DEFAULT_IMAGE_TOKEN, '').strip()
    if model.config.mm_use_im_start_end:
        question = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + question
    else:
        question = DEFAULT_IMAGE_TOKEN + '\n' + question

    prompt, conv = create_prompt(question)
    # print("prompt:", prompt)
    # print("conv:", conv)

    input_ids = (
        tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
    )
    # print("model.device:", model.device)
    # print("input_ids:", input_ids)

    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    # stopping_criteria = KeywordsStoppingCriteria(keywords=[stop_str], tokenizer=tokenizer, input_ids=input_ids)

    # taken from LLava-Med/llava/eval/model_vqa and the arguments provided in the readme
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor.unsqueeze(0).half().cuda(),
            do_sample=False,
            temperature=0,
            top_p=None,
            num_beams=None,
            # no_repeat_ngram_size=3,
            max_new_tokens=1024,
            use_cache=True,
        )

    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
    return outputs

In [8]:
def validate(data_path, image_base_path, model, tokenizer, image_processor, output_path):
  list_data_dict = json.load(open(data_path))
  results = []
  for data in list_data_dict:
      study_id = data["id"]
      image_url = os.path.join(image_base_path, data["image"])
      print("image_url:", image_url)
      conv = data["conversations"]
      question = conv[0]["value"]
      ground_truth = conv[1]["value"]
      prediction = get_prediction(
          model=model,
          tokenizer=tokenizer,
          image_processor=image_processor,
          image_url=image_url,
          question=question,
      )

      results.append(
        {
          "study_id": study_id,
          "prompt": question,
          "image": image_url, 
          "ground_truth": ground_truth, 
          "prediction": prediction
        }
      )

      print("ground_truth:", ground_truth, "prediction:", prediction)
      break
  # file = open(output_path, mode="w")
  # json.dump(results, file)
  # file.close()

In [8]:
# standard_prompt = "Analyze the following Chest X-ray image and provide your findings."÷
# prompt, conversation = create_prompt(standard_prompt)
# print("prompt", prompt)
# print("conversation", conversation)

In [9]:
# image = Image.open("./s50000014.jpg")
# image_tensor = process_images([image], image_processor, model.config)[0]

In [10]:
# image_url = "./s50000014.jpg"
# outputs = get_prediction(model=model, tokenizer=tokenizer, image_processor=image_processor, image_url=image_url, question=standard_prompt)

In [11]:
tokenizer, model, image_processor, context_len = load_base_model()

/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.04s/it]
Some weights of the model checkpoint at microsoft/llava-med-v1.5-mistral-7b were not used when initializing LlavaMistralForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.9.self_attn.v_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.4.self_attn.out_proj.weight', 'model.vision_tower.vision_tower.vision_model.post_layernorm.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.19.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.self_attn.k_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layer

In [20]:
print(model.get_model().vision_tower)

CLIPVisionTower(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELUActivation

In [ ]:
print(model.get_model().mm_projector)

In [10]:
# default settings trained on full dataset, with 4 bits quantization
finetuned_tokenizer, finetuned_model, finetuned_image_processor, finetuned_context_len  = load_model(
  model_path='../checkpoints/train_quantized_4_default',
  model_base='microsoft/llava-med-v1.5-mistral-7b',
  model_name='train_quantized_4_default'
)

/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:36<00:00,  9.19s/it]
Some weights of the model checkpoint at microsoft/llava-med-v1.5-mistral-7b were not used when initializing LlavaMistralForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.16.self_attn.out_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.9.self_attn.k_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.16.self_attn.q_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.15.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.e

Loading LoRA weights from ../checkpoints/train_quantized_4_default
Merging weights
Convert to FP16...


/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/transformers/modeling_utils.py:519: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(che

In [11]:
validate(
    data_path="/home/r11kaijun/MIMIC-CXR/processed_data/validate.json",
    image_base_path="/home/r11kaijun/physionet.org/files/mimic-cxr-jpg/2.1.0",
    model=finetuned_model,
    tokenizer=finetuned_tokenizer,
    image_processor=finetuned_image_processor,
    output_path="/home/r11kaijun/MIMIC-CXR/validation_results/train_quantized_4_default.json",
)

image_url: /home/r11kaijun/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p17/p17096560/s50056854/92846817-78ec0a2e-50265c40-0e7cad3b-6d912f6e.jpg


/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/r11kaijun/anaconda3/envs/llava-med/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:421: UserWarning: `num_beams` is set to None - defaulting to 1.
  warnings.warn("`num_beams` is set to None - defaulting to 1.", UserWarning)
The attention mask and the pad token id were not set. As a consequence, you ma

ground_truth: Left-sided subclavian port-a-cath continues to have its tip in the distal svc, near the cavoatrial junction. Heart remains enlarged, which may reflect cardiomegaly, although pericardial effusion should also be considered. Mediastinal contours are unchanged. There is persistent minimal blunting the left costophrenic angle, which may represent a tiny effusion or chronic pleural thickening. Streaky linear opacities at the left base may reflect areas of atelectasis or scarring. No focal airspace consolidation is seen to suggest pneumonia. No pneumothorax. No evidence of pulmonary edema. Linear density overlying the left upper quadrant is likely external to the patient, but clinical correlation would be recommended. prediction: .............................................................................................................................................................................................................................................................

In [ ]:
finetuned_tokenizer, finetuned_model, finetuned_image_processor, finetuned_context_len  = load_model(
  model_path='../checkpoints/train_5k-quantized-4-default',
  model_base='microsoft/llava-med-v1.5-mistral-7b',
  model_name='train_5k-quantized-4-default'
)

In [ ]:
validate(
    data_path="/home/r11kaijun/MIMIC-CXR/processed_data/validate.json",
    image_base_path="/home/r11kaijun/physionet.org/files/mimic-cxr-jpg/2.1.0",
    model=finetuned_model,
    tokenizer=finetuned_tokenizer,
    image_processor=finetuned_image_processor,
    output_path="train-5k-quantized-4-default.json",
)

In [ ]:
finetuned_tokenizer, finetuned_model, finetuned_image_processor, finetuned_context_len  = load_model(
  model_path='../checkpoints/train-with-indication-removed-history-5k-quantized-4-default',
  model_base='microsoft/llava-med-v1.5-mistral-7b',
  model_name='train-with-indication-removed-history-5k-quantized-4-default'
)

In [ ]:
validate(
    data_path="/home/r11kaijun/MIMIC-CXR/processed_data/validate_with_indication_removed_history.json",
    image_base_path="/home/r11kaijun/physionet.org/files/mimic-cxr-jpg/2.1.0",
    model=finetuned_model,
    tokenizer=finetuned_tokenizer,
    image_processor=finetuned_image_processor,
    output_path="train-with-indication-removed-history-5k-quantized-4-default.json",
)

### Obtain the labellings

In [1]:
# Run `pip uninstall numpy` and `pip install numpy==1.26.4`
from nltk.translate.bleu_score import sentence_bleu